<a href="https://colab.research.google.com/github/BergerZvika/Block-Breaker/blob/main/Copy_of_Image_Inpainting_with_GMCNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the current version of GMCNN pipeline from GitHub

In [ ]:
!git clone https://github.com/tlatkowski/inpainting-gmcnn-keras.git

fatal: destination path 'inpainting-gmcnn-keras' already exists and is not an empty directory.


In [ ]:
!ls

inpainting-gmcnn-keras	sample_data


# Download and extract NVIDIA's testing mask dataset

In [ ]:
!wget http://masc.cs.gmu.edu/wiki/uploads/partialconv/mask.zip
!unzip -q mask.zip

--2022-03-14 07:44:44--  http://masc.cs.gmu.edu/wiki/uploads/partialconv/mask.zip
Resolving masc.cs.gmu.edu (masc.cs.gmu.edu)... 129.174.125.37
Connecting to masc.cs.gmu.edu (masc.cs.gmu.edu)|129.174.125.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45168987 (43M) [application/zip]
Saving to: ‘mask.zip.2’

mask.zip.2          100%[===================>]  43.08M  12.2MB/s    in 6.7s    

2022-03-14 07:44:51 (6.39 MB/s) - ‘mask.zip.2’ saved [45168987/45168987]

replace mask/readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
!ls

# Download and extract dataset with training images (Places356)

In [ ]:
!git clone https://github.com/adielet/deep_learning_data.git
!mkdir images 
!cp -a deep_learning_data/photo_jpg/ images/val_large

fatal: destination path 'deep_learning_data' already exists and is not an empty directory.
mkdir: cannot create directory ‘images’: File exists


In [ ]:
!ls

deep_learning_data  images  inpainting-gmcnn-keras  mask  mask.zip  sample_data


# Install all requirements

In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install tensorflow-gpu==1.15.0
!pip install -r inpainting-gmcnn-keras/requirements/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2022-03-14 07:48:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  13.6MB/s    in 1.0s    

2022-03-14 07:48:09 (13.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   


In [ ]:
LOG_DIR = './outputs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://6100-34-140-109-46.ngrok.io


In [ ]:
!mkdir config
!cp inpainting-gmcnn-keras/config/main_config.ini config

mkdir: cannot create directory ‘config’: File exists


In [ ]:
%%writefile config/main_config.ini
[TRAINING]
WGAN_TRAINING_RATIO = 1
NUM_EPOCHS = 5
BATCH_SIZE = 4
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CHANNELS = 3
LEARNING_RATE = 0.0001
SAVE_MODEL_STEPS_PERIOD = 1000

[MODEL]
ADD_MASK_AS_GENERATOR_INPUT = True
GRADIENT_PENALTY_LOSS_WEIGHT = 10
ID_MRF_LOSS_WEIGHT = 0.05
ADVERSARIAL_LOSS_WEIGHT = 0.001
NN_STRETCH_SIGMA = 0.5
VGG_16_LAYERS = 3,6,10
ID_MRF_STYLE_WEIGHT = 1.0
ID_MRF_CONTENT_WEIGHT = 1.0
NUM_GAUSSIAN_STEPS = 3
GAUSSIAN_KERNEL_SIZE = 32
GAUSSIAN_KERNEL_STD = 40.0

Overwriting config/main_config.ini


In [ ]:
!ls deep_learning_data/photo_jpg



monet_jpg  photo_jpg


# Train generator with only confidence reconstruction loss for 5 epochs

In [ ]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask --experiment_name "gmcnn256x256" -warm_up_generator

Using TensorFlow backend.



INFO:tensorflow:Setting visible GPU to 0
INFO:tensorflow:Performing generator training only with the reconstruction loss.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
multiply_1 (Multiply)           (None, 256, 256, 3)  0           input_1[0][0]                    
                                                                 input_2[0][0]                    
______________________________________________________

# Visualize predicted images for specific training steps in warm-up generator mode

In [ ]:
!ls outputs/gmcnn256x256/predicted_pics/warm_up_generator/

step_000.png  step_1000.png


In [ ]:
from IPython.display import Image
Image('outputs/gmcnn256x256/predicted_pics/warm_up_generator/step_3000.png')

In [ ]:
Image('outputs/gmcnn256x256/predicted_pics/warm_up_generator/step_5000.png')

# Full Wasserstein GAN training mode: generator, local and global discriminators

In [ ]:
%%writefile config/main_config.ini
[TRAINING]
WGAN_TRAINING_RATIO = 5
NUM_EPOCHS = 5
BATCH_SIZE = 4
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CHANNELS = 3
LEARNING_RATE = 0.0002
SAVE_MODEL_STEPS_PERIOD = 500

[MODEL]
ADD_MASK_AS_GENERATOR_INPUT = True
GRADIENT_PENALTY_LOSS_WEIGHT = 10
ID_MRF_LOSS_WEIGHT = 0.05
ADVERSARIAL_LOSS_WEIGHT = 0.0005
NN_STRETCH_SIGMA = 0.5
VGG_16_LAYERS = 3,6,10
ID_MRF_STYLE_WEIGHT = 1.0
ID_MRF_CONTENT_WEIGHT = 1.0
NUM_GAUSSIAN_STEPS = 3
GAUSSIAN_KERNEL_SIZE = 32
GAUSSIAN_KERNEL_STD = 40.0

Overwriting config/main_config.ini


In [ ]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask -from_weights --experiment_name "gmcnn256x256"

Using TensorFlow backend.



INFO:tensorflow:Setting visible GPU to 0



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
multiply_1 (Multiply)           (None, 256, 256, 3)  0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (Non

# Vizualise results of full model training

In [ ]:
!ls outputs/gmcnn256x256/predicted_pics/wgan/

In [ ]:
Image('outputs/gmcnn256x256/predicted_pics/wgan/step_1000.png')

In [ ]:
Image('outputs/gmcnn256x256/predicted_pics/wgan/step_2000.png')

# Create zip file with model results and checkpoints

In [ ]:
!zip -r outputs.zip outputs

In [ ]:
ls

In [ ]:
!rm -rf outputs/

In [ ]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask -warm_up_generator -from_weights

In order to download model result go to: Files -> content, right click on outputs.zip -> Download